In [1]:
import os
from langchain_aws import ChatBedrock
from bs4 import BeautifulSoup
import pandas as pd
import subprocess
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

llm = ChatBedrock(
    model_id="us.anthropic.claude-3-haiku-20240307-v1:0",
    region = "us-east-1",    
    model_kwargs=dict(temperature=0)
)



In [2]:
from extractor.cdti.cdti_aid_extractor import CDTIAidExtractor

matrix_data = pd.read_csv('data/cdti/matrix_data/cdti_matrix_data.csv')
aid_matrix_data = matrix_data.iloc[0]
aid_matrix_data

instrument       Ayuda parcialmente reembolsable
support_ambit         Investigación y desarrollo
name                          Proyectos de I + D
url                 https://www.cdti.es/node/115
Name: 0, dtype: object

In [3]:
cdti_aid_extractor = CDTIAidExtractor(
    instrument=aid_matrix_data['instrument'],
    support_ambit=aid_matrix_data['support_ambit'],
    name=aid_matrix_data['name'],
    url=aid_matrix_data['url']
)
print(cdti_aid_extractor.parse_string(cdti_aid_extractor.name), cdti_aid_extractor.persist_data_dir)

proyectos_de_i_+_d data/cdti/cdti-aids/proyectos_de_i_+_d


In [4]:
aid_data = cdti_aid_extractor.run_aid_extraction(formated=True)
aid_data

extracting aid: Proyectos de I + D - Ayuda parcialmente reembolsable, Investigación y desarrollo
extracted title: Proyectos de I + D
extracted description: 2475
extracted card: 4171
extracted aid_doc:  https://www.cdti.es/sites/default/files/2025-03/proyectos_de_id.pdf


{'description_title': 'Proyectos de I + D',
 'description_body': '# Proyectos de I + D\nLos Proyectos de I+D son ayudas a proyectos de I+D desarrollados por empresas y destinados a la creación y mejora significativa de procesos productivos, productos o servicios.\n  Se presentan en distintas modalidades:\nProyectos de I+D individuales: presentados por una única empresa.\nProyectos de I+D de Cooperación Nacional: presentados por un consorcio de un mínimo de dos y un máximo de seis empresas autónomas. Deberán suscribir un acuerdo privado de colaboración que rija el consorcio.\nProyectos de I+D de Cooperación Tecnológica Internacional: presentados por empresas españolas participantes en programas de cooperación tecnológica internacional gestionados por el CDTI:\nProgramas multilaterales EUREKA, IBEROEKA y PRIMA.\nProgramas bilaterales establecidos por CDTI con Instituciones y Agencias de Financiación de otros países en virtud de acuerdos bilaterales.\nPrograma de proyectos internacionales

In [5]:
cdti_aid_extractor.persist_data_dir

'data/cdti/cdti-aids/proyectos_de_i_+_d'

In [ ]:
# def persist_data(self, aid_data):
# persist data

def persist_data(self, aid_data):

    if not os.path.exists(self.persist_data_dir):
        os.makedirs(self.persist_data_dir)

    # save description
    description_dir = self.persist_data_dir + "/" + self.parse_string(aid_data['description_title'] + "-description.md")
    # description_dir
    with open(description_dir, "w", encoding="utf-8") as file:
        file.write(aid_data['description_body'])

    # save card
    card_dir = self.persist_data_dir + "/" + self.parse_string(aid_data['description_title'] + "-card.md")
    # card_dir
    with open(card_dir, "w", encoding="utf-8") as file:
        file.write(aid_data['aid_card'])

    # save doc
    doc_name = aid_data['aid_doc_url'].split('/')[-1]
    doc_dir = self.persist_data_dir + "/" + doc_name
    doc_dir = doc_dir.replace("/", "\\")
    # doc_name

    powershell_script = f'''Invoke-WebRequest -uri "{aid_data['aid_doc_url']}" -OutFile "{doc_dir}"'''
    subprocess.run(["powershell", "-Command", powershell_script])



CompletedProcess(args=['powershell', '-Command', '\n$Url = "https://www.cdti.es/sites/default/files/2025-03/proyectos_de_id.pdf"\n$Destino = "data/cdti/cdti-aids/proyectos_de_i_+_d/proyectos_de_id.pdf"\nInvoke-WebRequest -Uri $Url -OutFile $Destino\n\n'], returncode=0)

In [18]:
# save doc
doc_name = aid_data['aid_doc_url'].split('/')[-1]
doc_dir = cdti_aid_extractor.persist_data_dir + "/" + doc_name
doc_dir = doc_dir.replace("/", "\\")
# doc_name

powershell_script = f'''Invoke-WebRequest -uri "{aid_data['aid_doc_url']}" -OutFile "{doc_dir}"'''
subprocess.run(["powershell", "-Command", powershell_script])

CompletedProcess(args=['powershell', '-Command', 'Invoke-WebRequest -uri "https://www.cdti.es/sites/default/files/2025-03/proyectos_de_id.pdf" -OutFile "data\\cdti\\cdti-aids\\proyectos_de_i_+_d\\proyectos_de_id.pdf"'], returncode=0)

In [16]:
doc_dir.replace("/", "\\")

'data\\cdti\\cdti-aids\\proyectos_de_i_+_d\\conv.pdf'

In [7]:
aid_data['aid_doc_url']


'https://www.cdti.es/sites/default/files/2025-03/proyectos_de_id.pdf'

In [66]:
from extractor.core.base_extractor import BaseExtractor
from extractor.params.extraction_params import (
    CDTI_MAIN_DIR, 
    IMPLICITY_WAIT_TIME
)

class CDTIAidExtractor(BaseExtractor):

    def __init__(
        self, 
        instrument,
        support_ambit,
        name,
        url):

        super().__init__()
        self.instrument=instrument
        self.support_ambit=support_ambit
        self.name=name
        self.url=url
        self.persist_data_dir = self.persist_data_dir + "/" + CDTI_MAIN_DIR + "/" + self.get_name_parsed()

        self.implicity_wait_time = IMPLICITY_WAIT_TIME


    def get_name_parsed(self):
        return self.name.replace(" ", "-").lower()

    
    def extract_description(self, url = None):

        url = url if url else self.url
        
        # driver = self.setup_driver()
        self.driver.get(url)
        # driver.implicitly_wait(self.implicity_wait_time)

        try:
            aid_title = self.driver.find_element(By.CLASS_NAME, "block-field-blocknodeayudastitle")
        except: 
            aid_title = None
        try:
            aid_description = self.driver.find_element(By.CLASS_NAME, "block-field-blocknodeayudasbody")
        except:
            aid_description = None

        if aid_description:
            aid_title_text = aid_title.text
            aid_description_text = aid_description.text  
            aid_description_md = "\n".join([chunk.strip() for chunk in aid_description_text.split("\n")])
            aid_description_md = "# " + aid_title.text + "\n" + aid_description_text

            # driver.quit()

            return aid_title_text, aid_description_md

        else:
            # driver.quit()
            return None, None


    
    def extract_aid_card(self, url=None):

        url = url if url else self.url

        # driver = self.setup_driver()
        self.driver.get(url)
        # driver.implicitly_wait(self.implicity_wait_time)
        try:
            aid_card_element = self.driver.find_element(By.CLASS_NAME, "card-body")
        except:
            aid_card_element = None

        if aid_card_element:
        
            aid_card_md = ""
            card_fields = aid_card_element.find_elements(By.CLASS_NAME, "ficha-field-wrapper")

            for card_field in card_fields:
                field_title = card_field.find_element(By.CLASS_NAME, "title").text.strip()
                field_text = card_field.find_element(By.CLASS_NAME, "text").text.strip()
                aid_card_md += f"# {field_title}\n{field_text}\n\n"

            # driver.quit()

            return aid_card_md

        else:
            # driver.quit()
            return None
        

    def extract_aid_doc_url(self, url=None):

        url = url if url else self.url

        # driver = self.setup_driver()
        self.driver.get(url)
        # driver.implicitly_wait(self.implicity_wait_time)

        try:
            aid_card_element = self.driver.find_element(By.CLASS_NAME, "card-body-normativa")
        except:
            aid_card_element = None
            
        if aid_card_element:
            a_aid = aid_card_element.find_element(By.TAG_NAME, "a")
            aid_url = a_aid.get_attribute("href")

            # driver.quit()
            return aid_url
        else:
            # driver.quit()
            return None


    def extract_aid_subpage(self, url=None):

        url = url if url else self.url

        # driver = self.setup_driver()
        self.driver.get(url)
        # driver.implicitly_wait(self.implicity_wait_time)

        try:
            aid_card_element = self.driver.find_element(By.CLASS_NAME, "view-display-id-block_1")
        except:
            aid_card_element = None

        if aid_card_element:
            try:
                a_element = aid_card_element.find_elements(By.TAG_NAME, "a")
            except:
                a_element = None

            subpage_name = a_element[0].text
            subpage_url = a_element[0].get_attribute("href")

            # driver.quit()

            return subpage_name, subpage_url

        else:
            # driver.quit()
            return None, None



    def run_aid_extraction(self, formated=True):

        extraction_data = {}

        self.driver = cdti_aid_extractor.setup_driver()

        print(f"extracting aid: {self.name} - {self.instrument}, {self.support_ambit}")
        description_title, description_body = self.extract_description()

        if description_title:
            print("extracted title:", description_title)
            extraction_data['description_title'] = description_title
        else:
            print("No description title found")
        
        if description_body:
            print("extracted description:", len(description_body))
            extraction_data['description_body'] =  description_body
        else:
            print("No description body found")

        # extract card
        aid_card = self.extract_aid_card()

        if aid_card:
            print("extracted card:", len(aid_card))
            extraction_data['aid_card'] = aid_card
        else:
            print("No card found")

        # extract doc url
        aid_doc_url = self.extract_aid_doc_url()

        if aid_doc_url:
            print("extracted aid_doc: ", aid_doc_url)
            extraction_data['aid_doc_url'] = aid_doc_url
        else:
            print("No doc url found")
            print("Extracting subpage")

            # extract subpage
            subpage_name, subpage_url = self.extract_aid_subpage()

            if subpage_name:
                print(f"found subpage: {subpage_name}, {subpage_url}")
                extraction_data['subpage'] = {
                    "subpage_name": subpage_name,
                    "subpage_url": subpage_url
                }

                print("extracting subpage")
                description_title_subpage, description_body_subpage = self.extract_description(url=subpage_url)

                if description_title_subpage:
                    print("extracted subpage title:", description_title_subpage)
                    extraction_data['subpage']['description_title_subpage'] = description_title_subpage
                else:
                    print("No title subpage found")

                if description_body_subpage:
                    print("extracted subpage description:", len(description_body_subpage))
                    extraction_data['subpage']['description_body_subpage'] = description_body_subpage
                else:
                    print("No description subpage found")

                # extract card
                aid_card_subpage = self.extract_aid_card(url=subpage_url)

                if aid_card_subpage:
                    print("extracted card:", len(aid_card_subpage))
                    extraction_data['subpage']['aid_card_subpage'] = aid_card_subpage
                else:
                    print("No card subpage found")

                # extract doc url
                aid_doc_subpage_url = self.extract_aid_doc_url(url=subpage_url)
                
                if aid_doc_subpage_url:
                    print("extracted aid_doc: ", aid_doc_subpage_url)
                    extraction_data['subpage']['aid_doc_subpage_url'] = aid_doc_subpage_url
                else:
                    print("No doc subpage url found")
            
            else:
                print("no subpage found")

        self.driver.quit()

        if formated:
            extraction_data = self.format_data(extraction_data)

        return extraction_data


    def format_data(self, aid_data):

        aid_data_formated = {}

        check_description_title = 'description_title' in aid_data.keys()
        check_description_body = 'description_body' in aid_data.keys()
        check_aid_card = 'aid_card' in aid_data.keys()
        check_aid_doc_url = 'aid_doc_url' in aid_data.keys()
        check_subpage = 'subpage' in aid_data.keys()
        check_description_title_subpage = ('description_title_subpage' in aid_data['subpage'].keys()) if check_subpage else False
        check_description_body_subpage = ('description_body_subpage' in aid_data['subpage'].keys()) if check_subpage else False
        check_aid_card_subpage = ('aid_card_subpage' in aid_data['subpage'].keys()) if check_subpage else False
        check_aid_doc_subpage_url = ('aid_doc_subpage_url' in aid_data['subpage'].keys()) if check_subpage else False

        # title
        if check_description_title:
            aid_data_formated['description_title'] = aid_data['description_title']
        elif not check_description_title and check_description_title_subpage:
            aid_data_formated['description_title'] = aid_data['subpage']['description_title_subpage']
        else:
            aid_data_formated['description_title'] = ''

        #  body
        if check_description_body and not check_description_body_subpage:
            aid_data_formated['description_body'] = aid_data['description_body']
        elif not check_description_body and check_description_body_subpage:
            aid_data_formated['description_body'] = aid_data['subpage']['description_body_subpage']
        elif check_description_body and check_description_body_subpage:
            aid_data_formated['description_body'] = aid_data['description_body'] + "\n\n" + aid_data['subpage']['description_body_subpage']
        else:
            aid_data_formated['description_body'] = ''

        # aid_card
        if check_aid_card:
            aid_data_formated['aid_card'] = aid_data['aid_card']
        elif not check_aid_card and check_aid_card_subpage:
            aid_data_formated['aid_card'] = aid_data['subpage']['aid_card_subpage']
        else:
            aid_data_formated['aid_card'] = ''

        # aid_doc_url
        if check_aid_doc_url:
            aid_data_formated['aid_doc_url'] = aid_data['aid_doc_url']
        if not check_aid_doc_url and check_aid_doc_subpage_url:
            aid_data_formated['aid_doc_url'] = aid_data['subpage']['aid_doc_subpage_url']
        else:
            aid_data_formated['aid_doc_url'] = ''

        return aid_data_formated


In [67]:
matrix_data = pd.read_csv('data/cdti/matrix_data/cdti_matrix_data.csv')
aid_data = matrix_data.iloc[1]
aid_data

instrument       Ayuda parcialmente reembolsable
support_ambit         Investigación y desarrollo
name              Actuación conjunta ISCIII-CDTI
url                 https://www.cdti.es/node/180
Name: 1, dtype: object

In [68]:
cdti_aid_extractor = CDTIAidExtractor(
    instrument=aid_data['instrument'],
    support_ambit=aid_data['support_ambit'],
    name=aid_data['name'],
    url=aid_data['url']
)
print(cdti_aid_extractor.get_name_parsed(), cdti_aid_extractor.persist_data_dir)


actuación-conjunta-isciii-cdti data/cdti/actuación-conjunta-isciii-cdti


In [69]:
aid_data = cdti_aid_extractor.run_aid_extraction()

extracting aid: Actuación conjunta ISCIII-CDTI - Ayuda parcialmente reembolsable, Investigación y desarrollo
extracted title: Actuación conjunta ISCIII-CDTI
extracted description: 1358
No card found
No doc url found
Extracting subpage
found subpage: ACTUACIÓN CONJUNTA ISCIII-CDTI 2023, https://www.cdti.es/ayudas/actuacion-conjunta-isciii-cdti-2023
extracting subpage
extracted subpage title: Actuación conjunta ISCIII-CDTI 2023
extracted subpage description: 413
extracted card: 3803
extracted aid_doc:  https://www.cdti.es/sites/default/files/2023-09/ficha_apr_mision_salud_2023.pdf


In [70]:
aid_data

{'description_title': 'Actuación conjunta ISCIII-CDTI',
 'description_body': '# Actuación conjunta ISCIII-CDTI\nEl Consejo de Ministros del 30 de noviembre de 2021 aprobó el Proyecto Estratégico para la Recuperación y Transformación Económica para la Salud de Vanguardia (PERTE de Salud), que se concibe como una propuesta país de interés general para el conjunto de la economía y para la sociedad.\nCon el PERTE de Salud se pretende impulsar el empleo de calidad y mejorar la salud de la población española a través del conocimiento científico, la innovación sanitaria de vanguardia y los datos como vectores de transformación.\nEn el marco de este PERTE, las actuaciones conjuntas entre el Instituto de Salud Carlos III (ISCIII) y el Centro para el Desarrollo Tecnológico y la Innovación (CDTI) persiguen coordinar las capacidades científicas con las capacidades empresariales para avanzar más rápidamente en el desarrollo de productos de diagnóstico clínico para la implementación de medicina pers

In [ ]:
aid_data_formated = cdti_aid_extractor




In [59]:
aid_data


{'description_title': 'Actuación conjunta ISCIII-CDTI',
 'description_body': '# Actuación conjunta ISCIII-CDTI\nEl Consejo de Ministros del 30 de noviembre de 2021 aprobó el Proyecto Estratégico para la Recuperación y Transformación Económica para la Salud de Vanguardia (PERTE de Salud), que se concibe como una propuesta país de interés general para el conjunto de la economía y para la sociedad.\nCon el PERTE de Salud se pretende impulsar el empleo de calidad y mejorar la salud de la población española a través del conocimiento científico, la innovación sanitaria de vanguardia y los datos como vectores de transformación.\nEn el marco de este PERTE, las actuaciones conjuntas entre el Instituto de Salud Carlos III (ISCIII) y el Centro para el Desarrollo Tecnológico y la Innovación (CDTI) persiguen coordinar las capacidades científicas con las capacidades empresariales para avanzar más rápidamente en el desarrollo de productos de diagnóstico clínico para la implementación de medicina pers

In [60]:
format_data(aid_data)

{'description_title': 'Actuación conjunta ISCIII-CDTI',
 'description_body': '# Actuación conjunta ISCIII-CDTI\nEl Consejo de Ministros del 30 de noviembre de 2021 aprobó el Proyecto Estratégico para la Recuperación y Transformación Económica para la Salud de Vanguardia (PERTE de Salud), que se concibe como una propuesta país de interés general para el conjunto de la economía y para la sociedad.\nCon el PERTE de Salud se pretende impulsar el empleo de calidad y mejorar la salud de la población española a través del conocimiento científico, la innovación sanitaria de vanguardia y los datos como vectores de transformación.\nEn el marco de este PERTE, las actuaciones conjuntas entre el Instituto de Salud Carlos III (ISCIII) y el Centro para el Desarrollo Tecnológico y la Innovación (CDTI) persiguen coordinar las capacidades científicas con las capacidades empresariales para avanzar más rápidamente en el desarrollo de productos de diagnóstico clínico para la implementación de medicina pers

In [31]:
aid_data_model = aid_data
aid_data_model

{'description': {'description_title': 'Proyectos de I + D',
  'description_body': '# Proyectos de I + D\nLos Proyectos de I+D son ayudas a proyectos de I+D desarrollados por empresas y destinados a la creación y mejora significativa de procesos productivos, productos o servicios.\n  Se presentan en distintas modalidades:\nProyectos de I+D individuales: presentados por una única empresa.\nProyectos de I+D de Cooperación Nacional: presentados por un consorcio de un mínimo de dos y un máximo de seis empresas autónomas. Deberán suscribir un acuerdo privado de colaboración que rija el consorcio.\nProyectos de I+D de Cooperación Tecnológica Internacional: presentados por empresas españolas participantes en programas de cooperación tecnológica internacional gestionados por el CDTI:\nProgramas multilaterales EUREKA, IBEROEKA y PRIMA.\nProgramas bilaterales establecidos por CDTI con Instituciones y Agencias de Financiación de otros países en virtud de acuerdos bilaterales.\nPrograma de proyecto

In [14]:
# extract description
cdti_aid_extractor.driver = cdti_aid_extractor.setup_driver()
# cdti_aid_extractor.driver.get(url)
# cdti_aid_extractor.driver.implicitly_wait(self.implicity_wait_time)


extraction_data = {}

print(f"extracting aid: {cdti_aid_extractor.name} - {cdti_aid_extractor.instrument}, {cdti_aid_extractor.support_ambit}")
description_title, description_body = cdti_aid_extractor.extract_description()

if description_title:
    print("extracted description:", len(description_body))
    extraction_data['description'] = {
        'description_title': description_title,
        'description_body': description_body
    }

else:
    print("No description found")

# extract card
aid_card = cdti_aid_extractor.extract_aid_card()

if aid_card:
    print("extracted card:", len(aid_card))
    extraction_data['aid_card'] = aid_card
else:
    print("No card found")

# extract doc url
aid_doc_url = cdti_aid_extractor.extract_aid_doc_url()
if aid_doc_url:
    print("extracted aid_doc: ", aid_doc_url)
    extraction_data['aid_doc_url'] = aid_doc_url

else:
    print("No doc url found")
    print("Extracting subpage")

    # extract subpage
    subpage_name, subpage_url = cdti_aid_extractor.extract_aid_subpage()

    if subpage_name:
        print(f"found subpage: {subpage_name}, {subpage_url}")
        extraction_data['subpage'] = {
            "subpage_name": subpage_name,
            "subpage_url": subpage_url
        }

        print("extracting subpage")
        description_title_subpage, description_body_subpage = cdti_aid_extractor.extract_description(url=subpage_url)
        if description_title_subpage:
            print("extracted description:", len(description_body_subpage))
            extraction_data['subpage']['description_title_subpage'] = description_title_subpage
            extraction_data['subpage']['description_body_subpage'] = description_body_subpage

        else:
            print("No description subpage found")

        # extract card
        aid_card_subpage = cdti_aid_extractor.extract_aid_card(url=subpage_url)
        if aid_card_subpage:
            print("extracted card:", len(aid_card_subpage))
            extraction_data['subpage']['aid_card_subpage'] = aid_card_subpage
        else:
            print("No card subpage found")

        # extract doc url
        aid_doc_subpage_url = cdti_aid_extractor.extract_aid_doc_url(url=subpage_url)
        if aid_doc_subpage_url:
            print("extracted aid_doc: ", aid_doc_subpage_url)
            extraction_data['subpage']['aid_doc_subpage_url'] = aid_doc_subpage_url
        else:
            print("No doc subpage url found")

cdti_aid_extractor.driver.quit()


extracting aid: Actuación conjunta ISCIII-CDTI - Ayuda parcialmente reembolsable, Investigación y desarrollo
extracted description: 1358
No card found
No doc url found
Extracting subpage
found subpage: ACTUACIÓN CONJUNTA ISCIII-CDTI 2023, https://www.cdti.es/ayudas/actuacion-conjunta-isciii-cdti-2023
extracting subpage
extracted description: 413
extracted card: 3803
extracted aid_doc:  https://www.cdti.es/sites/default/files/2023-09/ficha_apr_mision_salud_2023.pdf


In [15]:
# TODO: crear metodo en clase

# formateo de datos de salida para que el contenido de subpage se meta dentro del main
# si hay description en main y subpage, concatenarlo
# lo que no haya en main y si en subpage, llevarselo ahi
# el resultado tiene que ser:
# {'description': {'description_title': '',
#   'description_body': ''},
#  'aid_card': '',
#  'aid_doc_url': ''}

# persistencia de datos
# crear los ficheros con la info, texto plano en md y pdfs para los docs. buscar forma de
# descargar en el directorio el pdf desde python

# optimizar el scraping para que solo se abra una vez el driver y se cierre tras la extraccion.abs
# habria que lanzarlo desde fuera y llamarlo desde self internamente o pasarlo como param

extraction_data

{'description': {'description_title': 'Actuación conjunta ISCIII-CDTI',
  'description_body': '# Actuación conjunta ISCIII-CDTI\nEl Consejo de Ministros del 30 de noviembre de 2021 aprobó el Proyecto Estratégico para la Recuperación y Transformación Económica para la Salud de Vanguardia (PERTE de Salud), que se concibe como una propuesta país de interés general para el conjunto de la economía y para la sociedad.\nCon el PERTE de Salud se pretende impulsar el empleo de calidad y mejorar la salud de la población española a través del conocimiento científico, la innovación sanitaria de vanguardia y los datos como vectores de transformación.\nEn el marco de este PERTE, las actuaciones conjuntas entre el Instituto de Salud Carlos III (ISCIII) y el Centro para el Desarrollo Tecnológico y la Innovación (CDTI) persiguen coordinar las capacidades científicas con las capacidades empresariales para avanzar más rápidamente en el desarrollo de productos de diagnóstico clínico para la implementación

In [64]:
print(description_body_subpage)

# Actuación conjunta ISCIII-CDTI 2023
El CDTI concederá ayudas parcialmente reembolsables a los proyectos de I+D desarrollados por empresas en cooperación con centros y organismos de investigación en el ámbito de la medicina personalizada y terapias emergentes.
Los proyectos deberán encajar en la misión definida, sobre terapias avanzadas y medicina de precisión enfocadas a:
Envejecimiento
Enfermedades crónicas


In [21]:
print(cdti_aid_extractor.extract_aid_card())

None


In [50]:
# busqueda de url de normativa
print(cdti_aid_extractor.extract_aid_doc_url())

None


In [ ]:
cdti_aid_extractor.extract_aid_subpage()<

('ACTUACIÓN CONJUNTA ISCIII-CDTI 2023',
 'https://www.cdti.es/ayudas/actuacion-conjunta-isciii-cdti-2023')

In [ ]:
# extraccion de subconvocatoria
# view-id-ayudas
# view-display-id-block_1
driver = cdti_aid_extractor.setup_driver()
driver.get(cdti_aid_extractor.url)
driver.implicitly_wait(3)
try:
    aid_card_element = driver.find_element(By.CLASS_NAME, "view-display-id-block_1")
except:
    aid_card_element = None

try:
    a_element = aid_card_element.find_elements(By.TAG_NAME, "a")
except:
    a_element = None

print(a_element[0].text, a_element[0].get_attribute("href"))

ACTUACIÓN CONJUNTA ISCIII-CDTI 2023 https://www.cdti.es/ayudas/actuacion-conjunta-isciii-cdti-2023


In [ ]:


print(a_element[0].get_attribute("href"))

https://www.cdti.es/ayudas/actuacion-conjunta-isciii-cdti-2023


In [ ]:
try:
    a_element = aid_card_element.find_elements(By.TAG_NAME, "a")
except:
    a_element = None

In [16]:
print(cdti_aid_extractor.url)

https://www.cdti.es/node/115


In [28]:
driver = cdti_aid_extractor.driver
url = cdti_aid_extractor.url

driver.get(url)
driver.implicitly_wait(5)
# find matrix block
# div_element = driver.find_element(By.CLASS_NAME, "block-field-blocknodeayudasbody")

In [54]:
# aid description

aid_description = driver.find_element(By.CLASS_NAME, "block-field-blocknodeayudasbody").text
aid_description = "\n".join([chunk.strip() for chunk in aid_description.split("\n")])
print(aid_description)

Los Proyectos de I+D son ayudas a proyectos de I+D desarrollados por empresas y destinados a la creación y mejora significativa de procesos productivos, productos o servicios.
Se presentan en distintas modalidades:
Proyectos de I+D individuales: presentados por una única empresa.
Proyectos de I+D de Cooperación Nacional: presentados por un consorcio de un mínimo de dos y un máximo de seis empresas autónomas. Deberán suscribir un acuerdo privado de colaboración que rija el consorcio.
Proyectos de I+D de Cooperación Tecnológica Internacional: presentados por empresas españolas participantes en programas de cooperación tecnológica internacional gestionados por el CDTI:
Programas multilaterales EUREKA, IBEROEKA y PRIMA.
Programas bilaterales establecidos por CDTI con Instituciones y Agencias de Financiación de otros países en virtud de acuerdos bilaterales.
Programa de proyectos internacionales con certificación y seguimiento unilateral por CDTI.
Proyectos de I+D de Cooperación Tecnológica

In [ ]:
aid_card = {}

aid_card_element = driver.find_element(By.CLASS_NAME, "card-body")
card_fields = aid_card_element.find_elements(By.CLASS_NAME, "ficha-field-wrapper")
for card_field in card_fields:
    field_title = card_field.find_element(By.CLASS_NAME, "title").text.strip()
    field_text = card_field.find_element(By.CLASS_NAME, "text").text.strip()

    aid_card[str(field_title)] = field_text
    # print(field_title, "\n", field_text, "\n\n")
    aid_card += f"# {field_title}\n{field_text}\n\n"

print(aid_card)

# Actuación
Proyectos CDTI de I+D

# Objetivo general de la actuación
Ayudas a proyectos de I+D desarrollados por empresas y destinados a la creación y mejora significativa de procesos productivos, productos o servicios.
Pueden comprender tanto actividades de investigación industrial como de desarrollo experimental.
No existe ninguna restricción en cuanto al sector o a la tecnología a desarrollar.

# Beneficiarios
Empresas.

# Tipo de convocatoria
Convocatoria abierta todo el año.

# Plazo de presentación
Todo el año.

# Tipo de la ayuda
Ayuda parcialmente reembolsable.

# Características de la ayuda
Tipo de interés fijo: Euribor a 1 año.
Ayuda de hasta el 85% del presupuesto aprobado.
La empresa deberá aportar, al menos, el 15% del presupuesto del proyecto con recursos propios.
Anticipo de hasta el 50% de la ayuda con límite de 300.000 euros, sin exigencia de garantías adicionales.
Tramo no reembolsable de entre el 10% y el 33% de la ayuda.
Devolución: 10 o 15 años incluyendo una care

In [ ]:
# pdf convocatoria cuidaooo
# a veces viene como ficha del instrumento y a veces como listado de convocatorias, que hay que coger la primera
aid_call_document = driver.find_element(By.CLASS_NAME, "view-id-ayudas")
aid_call_document.text


'Ficha resumen de la ayuda\nActuación\nProyectos CDTI de I+D\nObjetivo general de la actuación\nAyudas a proyectos de I+D desarrollados por empresas y destinados a la creación y mejora significativa de procesos productivos, productos o servicios.\nPueden comprender tanto actividades de investigación industrial como de desarrollo experimental.\nNo existe ninguna restricción en cuanto al sector o a la tecnología a desarrollar.\nBeneficiarios\nEmpresas.\n  Tipo de convocatoria\nConvocatoria abierta todo el año.\nPlazo de presentación\nTodo el año.\nTipo de la ayuda\nAyuda parcialmente reembolsable. \nCaracterísticas de la ayuda\nTipo de interés fijo: Euribor a 1 año.\nAyuda de hasta el 85% del presupuesto aprobado.\nLa empresa deberá aportar, al menos, el 15% del presupuesto del proyecto con recursos propios.\nAnticipo de hasta el 50% de la ayuda con límite de 300.000 euros, sin exigencia de garantías adicionales.\nTramo no reembolsable de entre el 10% y el 33% de la ayuda.\nDevolución: 1

In [ ]:
'''
formato de enlaces

* normal: como i+d: en el enlace principal viene:
    * body
    * ficha
    * enlace a pdf: ficha del instrumento

* caso 1: Actuación conjunta ISCIII-CDTI: 
    * hay un body
    hay una lista de enlaces de convocatorias presentes y pasadas. Hay que coger la primera
    * en ese enlace:
        * otro body mas pequeño
        * ficha
        * enlace a pdf (ficha del instrumento)
* caso 2: Programa Tecnológico de Automoción Sostenible (PTAS):
    * no hay body, aunque podria haberlo
    * ficha
    * enlace a pdf: documentacion convocatorias
* caso 3: CDTI-Era-Net:
    * body
    * ficha
    * lista de enlaces a convocatorias. coger la primera.
        * otro body
        enlace a pdf: texto de la convocatoria
'''

In [ ]:
'''
mas o menos generico:

texto descripcion: 
block-field-blocknodeayudastitle
block-field-blocknodeayudasbody

ficha: card-body

enlace documento pdf: card-body-normativa

cuando hay varias opciones de convocatorias:
view-id-ayudas
view-display-id-block_1

'''

In [ ]:
'''
flujo del scrapper

entra en url desde matrix
busca title y body de descripcion. 
pillar title para nombre documento
Si existe el body, genera un md con el contenido
busca ficha del documento. Si exsite, genera un md con el contenido
busca enlace a pdf de convocatoria (card-body-normativa). 
if existe:
    lo guarda en pdf
else:
    busca enlace de subpaginas: 
        view-id-ayudas
        view-display-id-block_1
    Sacar las url y pillar la primera que es la mas actual.
    a esa url quizas haya que añadirle https://www.cdti.es/

    en esa pagina repetimos:
    busca title y body de descripcion. 
    pillar title para nombre documento
    Si existe el body, genera un md con el contenido
    busca ficha del documento. Si exsite, genera un md con el contenido

'''

In [ ]:
necesito un codigo en python con selenium que haga lo siguiente:

rellena un diccionario con los siguiente:
a partir de una url busca:
comprueba si existen los elementos de clase block-field-blocknodeayudastitle yblock-field-blocknodeayudasbody
En caso de existir, extrae el texto de ambos, los concatena y lo guarda en el diccionario con el nombre "aid_description_1"
Busca el elemento de clase card-body, y si existe, extrae todo el texto que contiene concatenado, de todos los elementos de clase ficha-field-wrapper. Los concatena y lo guarda en el diccionario con el nombre "aid_card_1"
ahora busca el elemento card-body-normativa
si existe, almacena la url en el diccionario con el nombre "aid_call_document_url"
si no existe:
comprueba si hay mas enlaces a subdocumentos. busca el elemento view-id-ayudas y extrae la primera url que aparece.

Con selenium, en esa url, repite el proceso de busqueda de los elementos block-field-blocknodeayudastitle, block-field-blocknodeayudasbody, card-body y card-body-normativa
En caso de existir cada uno, los almacena en json de la misma forma, pero empleando el sufijo _2

In [47]:
field_text

'Estas ayudas se ajustarán a lo establecido en el Reglamento (UE) n.º 651/2014, de la Comisión, de 17 de junio de 2014, por el que se declaran determinadas categorías de ayuda compatibles con el mercado interior en aplicación de los artículos 107 y 108 del Tratado (Reglamento general de exención por categorías).'

In [37]:
card_fields = aid_card.find_elements(By.CLASS_NAME, "ficha-field-wrapper")
for card_field in card_fields:
    field_title = card_field.find_element(By.CLASS_NAME, "title").text.strip()
    field_text = card_field.find_element(By.CLASS_NAME, "text").text.strip()
    print(field_title, "\n", field_text, "\n\n")


Actuación 
 Proyectos CDTI de I+D 


Objetivo general de la actuación 
 Ayudas a proyectos de I+D desarrollados por empresas y destinados a la creación y mejora significativa de procesos productivos, productos o servicios.
Pueden comprender tanto actividades de investigación industrial como de desarrollo experimental.
No existe ninguna restricción en cuanto al sector o a la tecnología a desarrollar. 


Beneficiarios 
 Empresas. 


Tipo de convocatoria 
 Convocatoria abierta todo el año. 


Plazo de presentación 
 Todo el año. 


Tipo de la ayuda 
 Ayuda parcialmente reembolsable. 


Características de la ayuda 
 Tipo de interés fijo: Euribor a 1 año.
Ayuda de hasta el 85% del presupuesto aprobado.
La empresa deberá aportar, al menos, el 15% del presupuesto del proyecto con recursos propios.
Anticipo de hasta el 50% de la ayuda con límite de 300.000 euros, sin exigencia de garantías adicionales.
Tramo no reembolsable de entre el 10% y el 33% de la ayuda.
Devolución: 10 o 15 años incluye

In [35]:
card_field = card_fields[0]
card_field.find_element(By.CLASS_NAME, "title").text
card_field.find_element(By.CLASS_NAME, "text").text

'Proyectos CDTI de I+D'

In [ ]:
url = "https://ejemplo.com"  # Reemplaza con la URL que necesitas
driver.get(url)

try:
    # Encontrar el div por clase
    div_element = driver.find_element(By.CLASS_NAME, "block-field-blocknodeayudasbody")
    
    # Obtener todos los elementos dentro del div
    inner_elements = div_element.find_elements(By.XPATH, ".//*")
    
    # Extraer y mostrar los textos
    for element in inner_elements:
        text = element.text.strip()
        if text:
            print(text)
finally:
    driver.quit()